### Sentiment analysis Doc2Vec

In [1]:
import gensim
from sklearn.model_selection import train_test_split  
import numpy as np
import glob
import os 
import imdb_preprocess

#### Read data

In [2]:
data_path = '../../imdb'
file_path = os.path.join(data_path,'imdb.p')
if os.path.exists(file_path): 
    print('load pickle data from disk')
    x,y = imdb_preprocess.load_processed_data(file_path)
else:
    print('process and load data')
    imdb_preprocess.save_processed_data(data_path)
    x,y = imdb_preprocess.load_processed_data(file_path)

load pickle data from disk


In [3]:
## train and test split data 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(len(x_train)," ",len(y_train))

20000   20000


### Gensim doc2vec model

Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
We do this by using the LabeledSentence method. The format will be "TRAIN_i" or "TEST_i" where "i" is
a dummy index of the review.

In [4]:
## use labeledSentence object
#LabeledSentence = gensim.models.doc2vec.LabeledSentence
from gensim.models.doc2vec import TaggedDocument
import random
from gensim.models import Doc2Vec

In [5]:
def read_corpus(reviews):
    for i,doc in enumerate(reviews):
        yield TaggedDocument(doc,[i])

train_corpus = list(read_corpus(x_train))

In [6]:
cores = os.cpu_count()
size = 200
window = 5 
iteration = 20 

model = Doc2Vec(size=size, min_count=2, window = window,iter=iteration)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

## after you trained your model, you can train them again 
# model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

74423963

In [7]:

## save model 
if not os.path.exists(os.path.join(data_path,"trained")):
    os.makedirs(os.path.join(data_path,"trained"))
    model.save(os.path.join(data_path,'trained','imdb.d2v'))
else:
    model.save(os.path.join(data_path,'trained','imdb.d2v'))
    model = Doc2Vec.load(os.path.join(data_path,'trained','imdb.d2v'))

### look at some samples 

In [12]:
model = Doc2Vec.load(os.path.join(data_path,'trained','imdb.d2v'))

In [13]:
model.docvecs.most_similar(positive=[1],topn=10)

[(5059, 0.5437087416648865),
 (17694, 0.5340926647186279),
 (7922, 0.5043013095855713),
 (17374, 0.5001096129417419),
 (5473, 0.496317982673645),
 (9279, 0.4929776191711426),
 (12234, 0.4925103783607483),
 (10386, 0.4923815131187439),
 (262, 0.48916536569595337),
 (10065, 0.48846209049224854)]

In [14]:
print(' '.join(x[1]) , '\n\n')
print(' '.join(x[4528]))

i had the tv on for a white noise companion and heard" $400 for a fully furnished apartment" so i ran into the tv room expecting another 70's flick and got much more . luckily , i could rewind to the beginning ( dvr buffer ) and hit the record button to watch it entirely . ( cinemax uncut and in hd no less ! ) aside from some holes in the story and intermittent improbable dialog/events , this is an effective thriller worthy of your time to watch . pretty creepy and progressive at times : beverly d'angelo's character masturbates in front of alison parker , played adroitly by cristina raines , parker stabs , in very gory fashion , her father , an explicit menage a trios scene . ( don't let the kids watch ) the film is totally 70's full of bad clothes ( polyester suits and tacky ascots ) and decor , bad hair , over bloated music score , and familiar looking cinematography . the cast is excellent , take a second on this film's home page to check it out . it was a surprise to see christophe

In [76]:
infer_review = 'This is obviously the best movie ever in my life !'
infer_vec = model.infer_vector(infer_review.split())
infer_vec.shape

(200,)

In [77]:
most_sim_index = model.docvecs.most_similar([infer_vec],topn=10)[0][0]
print(infer_review,'\n\n')
print(' '.join(x[most_sim_index]))

This is obviously the best movie ever in my life ! 


this is essentially a variation on house of wax , in both the plot and the type of role played by the star of both movies , vincent price . in both pictures he plays a talented artist who is sent toppling over the edge into insanity when his creations are usurped by other , less talented and less scrupulous people . in this movie he plays a designer of illusions for stage magicians who aspires to set out on a performing career himself only to be frustrated when another illusionist , the great rinaldo ( john emery ) insists that he honour his contract and give him first choice of any illusions he designs . price is already ill disposed towards rinaldi as his former wife is now a paramour of rinaldi . he deploys his talents as an illusionist and as a brilliant mimic to avenge himself upon rinaldi and others who thwart his plans for recognition as a performer and a designer . price is pretty much the whole show here and gives a well ju

### Use document vector for sentiment analysis

In [78]:
#Get training set vectors from our models
def getVecs(model, corpus, size):
    vecs = [model.docvecs[z].reshape((1, size)) for z in range(len(x_train))]
    return np.concatenate(vecs)

In [79]:
train_vecs_dm = getVecs(model,x_train,size)

In [80]:
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier(loss='log', penalty='l1')
lr.fit(train_vecs_dm,y_train )

print ('Test Accuracy: %.2f'%lr.score(train_vecs_dm, y_train))

Test Accuracy: 0.80
